# Importing Libraries

In [1]:
from imutils import paths
import numpy as np
import imutils
import pickle
import cv2
import os

In [5]:
dataset = r"C:\Users\HP.DESKTOP-5IHNLMQ\Desktop\VS code\Dataset"

#initial name for embedding file
embeddingFile = "output/embeddings.pickle"

#initializing model for embedding Pytorch
embeddingModel = "openface_nn4.small2.v1.t7"

#initialization of caffe model for face detection
prototxt = "model/deploy.prototxt"
model = "model/res10_300x300_ssd_iter_140000.caffemodel"

#loading caffe model for face detection
#detecting face from Image via Caffe deep learning
detector = cv2.dnn.readNetFromCaffe(prototxt, model)

#loading pytorch model file for extract facial embeddings
#extracting facial embeding via deep learning feature extraction
embedder = cv2.dnn.readNetFromTorch(embeddingModel)

#getting image paths
imagePaths = list(paths.list_images(dataset))

#initialization
knownEmbeddings = []
knownNames = []
total = 0
conf = 0.5

#we start to read images one by to apply face detection and embedding
for (i, imagePath) in enumerate(imagePaths):
    print("Preprocessing image {}/{}".format(i + 1, len(imagePaths)))
    name = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width= 600)
    (h, w) = image.shape[:2]
    #converting image to blob for dnn face detection
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300,300)), 1.0, (300,300), (104.0, 177.0, 123.0), swapRB = False, crop= False)
    
    #setting input blob image
    detector.setInput(imageBlob)
    #prediction the face
    detections = detector.forward()
    
    if len(detections) > 0:
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]
        
        if confidence > conf:
            #ROI range of interest
            box = detections[0, 0, i, 3:7] * np.array([w,h,w,h])
            (startX, startY, endX, endY) = box.astype('int')
            face = image[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]
            
            if fW < 20 or fH < 20:
                continue
                
            #image to blob for face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB = True, crop =False)
            #facial features embedder input image face blob
            embedder.setInput(faceBlob)
            vec = embedder.forward()
            knownNames.append(name)
            knownEmbeddings.append(vec.flatten())
            total += 1
            
print("Embedding:{}".format(total))
data = {"embeddings": knownEmbeddings, "names":knownNames}
f = open(embeddingFile, "wb")
f.write(pickle.dumps(data))
f.close()
print("Process Completed")

Preprocessing image 1/300
Preprocessing image 2/300
Preprocessing image 3/300
Preprocessing image 4/300
Preprocessing image 5/300
Preprocessing image 6/300
Preprocessing image 7/300
Preprocessing image 8/300
Preprocessing image 9/300
Preprocessing image 10/300
Preprocessing image 11/300
Preprocessing image 12/300
Preprocessing image 13/300
Preprocessing image 14/300
Preprocessing image 15/300
Preprocessing image 16/300
Preprocessing image 17/300
Preprocessing image 18/300
Preprocessing image 19/300
Preprocessing image 20/300
Preprocessing image 21/300
Preprocessing image 22/300
Preprocessing image 23/300
Preprocessing image 24/300
Preprocessing image 25/300
Preprocessing image 26/300
Preprocessing image 27/300
Preprocessing image 28/300
Preprocessing image 29/300
Preprocessing image 30/300
Preprocessing image 31/300
Preprocessing image 32/300
Preprocessing image 33/300
Preprocessing image 34/300
Preprocessing image 35/300
Preprocessing image 36/300
Preprocessing image 37/300
Preprocess

Preprocessing image 298/300
Preprocessing image 299/300
Preprocessing image 300/300
Embedding:300
Process Completed
